In [1]:
import sys
sys.path.insert(1, '../DraCooN/algorithm')

import os
import pandas as pd
from GRN_simulations import GRN_simulator
from tqdm import tqdm
import pickle


In [2]:
create_folders = ["simulations", "results"]
#network_types = ['DC', 'DR']
network_types = ['DR']
assessment_types = ['proportion_casecontrol',
                    'proportion_perturbed_genes',
                    'perturbation_noise',
                    'signal_noise',
                    'gene_number',
                   'ingroup_heterogeneity']

for folder in create_folders:
    if os.path.exists(folder):
        os.system(f"rm -rf {folder}")
    os.system(f"mkdir {folder}")

    for nettype in network_types:
        net_folder = os.path.join(folder, nettype)
        if os.path.exists(net_folder):
            os.system(f"rm -rf {net_folder}")
        os.system(f"mkdir {net_folder}")

        for assessment in assessment_types:
            assessment_folder = os.path.join(net_folder, assessment)
            if os.path.exists(assessment_folder):
                os.system(f"rm -rf {assessment_folder}")
            os.system(f"mkdir {assessment_folder}")


In [3]:
# What are the baseline parameters?
default_n_genes = 100
default_n_tfs = int(0.3 * default_n_genes)
default_n_samples = 200
default_mean_expression = 2
default_simdata_noise = 1

default_perturbation_noise = 1
default_proportion_group_case = 0.5
default_ingroup_perturbed_proportion = 1

default_affgenes = int(0.4 * default_n_genes)
default_shift_alpha = 2
default_nruns = 5

num, div = default_affgenes, 4
splitaffgenes = [num // div + (1 if x < num % div else 0) for x in range(div)]

default_perturbeds = [[default_affgenes, 0, 0, 0], 
              [0, default_affgenes, 0, 0],
              [0, 0, default_affgenes, 0],
              [0, 0, 0, default_affgenes],
              splitaffgenes]

# 1. Comparing perturbed and control group sizes

In [4]:
#Baseline parameters
which_assessment = 'proportion_casecontrol'

datainfo=0

proportion_case = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for proportion in tqdm(proportion_case):
    for p in range(len(default_perturbeds)):
       
        #print(perturbeds[p][0], perturbeds[p][1], perturbeds[p][2],  perturbeds[p][3])

        simulation = GRN_simulator(n_genes=default_n_genes, n_tfs=default_n_tfs, n_samples=default_n_samples,
                                   n_shutdowns=default_perturbeds[p][0], n_inversions=default_perturbeds[p][1], 
                                   n_shifts=default_perturbeds[p][2], n_uncorrelateds=default_perturbeds[p][3], 

                                   proportion_group_case=proportion,
                                   simdata_noise=default_simdata_noise, perturbation_noise=default_perturbation_noise, mean_expression=default_mean_expression,
                                   shift_alpha = default_shift_alpha, ingroup_perturbed_proportion=default_ingroup_perturbed_proportion, zscore_normalize = False)
        simulation.generate_DRN()

        simdrnet_rootname = os.path.join(f'simulations/DR/', which_assessment, f'propCC_{proportion}_perts_{default_perturbeds[p][0]}_{default_perturbeds[p][1]}_{default_perturbeds[p][2]}_{default_perturbeds[p][3]}_')
        #print(simdrnet_rootname)

        # Now, we'll save the object using pickle.
        with open(f'{simdrnet_rootname}.pkl', 'wb') as f:
            pickle.dump(simulation, f)


        simulation.simulated_data.T.to_csv(simdrnet_rootname + '_biomdata.csv')
        simulation.condition_data.to_csv(simdrnet_rootname + '_conddata.csv')
        simulation.refnet.to_csv(simdrnet_rootname + '_refnet.csv')
        simulation.structure.to_csv(simdrnet_rootname + '_structure.csv')
        datainfo += 1
print(f"Total number of {which_assessment} simulations: {datainfo}")



100%|███████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:20<00:00,  2.28s/it]

Total number of proportion_casecontrol simulations: 45


# 2. Comparing proportion of perturbed genes

In [5]:
which_assessment = 'proportion_perturbed_genes'



proportion_genes = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

datainfo = 0
for proportion in tqdm(proportion_genes):
    
    default_affgenes = int(proportion * default_n_genes)

    num, div = default_affgenes, 4
    splitaffgenes = [num // div + (1 if x < num % div else 0) for x in range(div)]

    default_perturbeds = [[default_affgenes, 0, 0, 0], 
                  [0, default_affgenes, 0, 0],
                  [0, 0, default_affgenes, 0],
                  [0, 0, 0, default_affgenes],
                  splitaffgenes]
    
    for p in range(len(default_perturbeds)):
        #print(default_perturbeds[p][0], default_perturbeds[p][1], default_perturbeds[p][2],  default_perturbeds[p][3])

        simulation = GRN_simulator(n_genes=default_n_genes, n_tfs=default_n_tfs, n_samples=default_n_samples,
                                   n_shutdowns=default_perturbeds[p][0], n_inversions=default_perturbeds[p][1], 
                                   n_shifts=default_perturbeds[p][2], n_uncorrelateds=default_perturbeds[p][3], 

                                   proportion_group_case=default_proportion_group_case,
                                   simdata_noise=default_simdata_noise, perturbation_noise=default_perturbation_noise, mean_expression=default_mean_expression,
                                   shift_alpha = default_shift_alpha, ingroup_perturbed_proportion=default_ingroup_perturbed_proportion, zscore_normalize = False)
        simulation.generate_DRN()

        simdrnet_rootname = os.path.join(f'simulations/DR/', which_assessment, f'propPertG_{proportion}_perts_{default_perturbeds[p][0]}_{default_perturbeds[p][1]}_{default_perturbeds[p][2]}_{default_perturbeds[p][3]}_')
        #print(simdrnet_rootname)

        # Now, we'll save the object using pickle.
        with open(f'{simdrnet_rootname}.pkl', 'wb') as f:
            pickle.dump(simulation, f)


        simulation.simulated_data.T.to_csv(simdrnet_rootname + '_biomdata.csv')
        simulation.condition_data.to_csv(simdrnet_rootname + '_conddata.csv')
        simulation.refnet.to_csv(simdrnet_rootname + '_refnet.csv')
        simulation.structure.to_csv(simdrnet_rootname + '_structure.csv')
        datainfo += 1
        
print(f"Total number of {which_assessment} simulations: {datainfo}")


100%|███████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:22<00:00,  2.49s/it]

Total number of proportion_perturbed_genes simulations: 45


# 3. Perturbation noise

In [6]:
which_assessment = 'perturbation_noise'


datainfo=0

perturb_noises = [0.1, 0.5, 1, 1.5, 2]

for proportion in tqdm(perturb_noises):
    for p in range(len(default_perturbeds)):
        #print(perturbeds[p][0], perturbeds[p][1], perturbeds[p][2],  perturbeds[p][3])

        simulation = GRN_simulator(n_genes=default_n_genes, n_tfs=default_n_tfs, n_samples=default_n_samples,
                                   n_shutdowns=default_perturbeds[p][0], n_inversions=default_perturbeds[p][1], 
                                   n_shifts=default_perturbeds[p][2], n_uncorrelateds=default_perturbeds[p][3], 

                                   proportion_group_case=default_proportion_group_case,
                                   simdata_noise=default_simdata_noise, perturbation_noise=proportion, mean_expression=default_mean_expression,
                                   shift_alpha = default_shift_alpha, ingroup_perturbed_proportion=default_ingroup_perturbed_proportion, zscore_normalize = False)
        simulation.generate_DRN()

        simdrnet_rootname = os.path.join(f'simulations/DR/', which_assessment, f'perturbnoise_{proportion}_perts_{default_perturbeds[p][0]}_{default_perturbeds[p][1]}_{default_perturbeds[p][2]}_{default_perturbeds[p][3]}_')
        #print(simdrnet_rootname)

        # Now, we'll save the object using pickle.
        with open(f'{simdrnet_rootname}.pkl', 'wb') as f:
            pickle.dump(simulation, f)


        simulation.simulated_data.T.to_csv(simdrnet_rootname + '_biomdata.csv')
        simulation.condition_data.to_csv(simdrnet_rootname + '_conddata.csv')
        simulation.refnet.to_csv(simdrnet_rootname + '_refnet.csv')
        simulation.structure.to_csv(simdrnet_rootname + '_structure.csv')
        datainfo += 1


print(f"Total number of {which_assessment} simulations: {datainfo}")


100%|███████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.35s/it]

Total number of perturbation_noise simulations: 25


# 4. Comparing signal noises

In [7]:
which_assessment = 'signal_noise'

datainfo=0

signal_noises = [0.1, 0.5, 1, 1.5, 2]

for proportion in tqdm(signal_noises):
    for p in range(len(default_perturbeds)):
        #print(perturbeds[p][0], perturbeds[p][1], perturbeds[p][2],  perturbeds[p][3])

        simulation = GRN_simulator(n_genes=default_n_genes, n_tfs=default_n_tfs, n_samples=default_n_samples,
                                   n_shutdowns=default_perturbeds[p][0], n_inversions=default_perturbeds[p][1], 
                                   n_shifts=default_perturbeds[p][2], n_uncorrelateds=default_perturbeds[p][3], 

                                   proportion_group_case=default_proportion_group_case,
                                   simdata_noise=proportion, perturbation_noise=default_perturbation_noise, mean_expression=default_mean_expression,
                                   shift_alpha = default_shift_alpha, ingroup_perturbed_proportion=default_ingroup_perturbed_proportion, zscore_normalize = False)
        simulation.generate_DRN()

        simdrnet_rootname = os.path.join(f'simulations/DR/', which_assessment, f'signalnoise_{proportion}_perts_{default_perturbeds[p][0]}_{default_perturbeds[p][1]}_{default_perturbeds[p][2]}_{default_perturbeds[p][3]}_')
        #print(simdrnet_rootname)

        # Now, we'll save the object using pickle.
        with open(f'{simdrnet_rootname}.pkl', 'wb') as f:
            pickle.dump(simulation, f)


        simulation.simulated_data.T.to_csv(simdrnet_rootname + '_biomdata.csv')
        simulation.condition_data.to_csv(simdrnet_rootname + '_conddata.csv')
        simulation.refnet.to_csv(simdrnet_rootname + '_refnet.csv')
        simulation.structure.to_csv(simdrnet_rootname + '_structure.csv')
        datainfo += 1

print(f"Total number of {which_assessment} simulations: {datainfo}")



100%|███████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.94s/it]

Total number of signal_noise simulations: 25


# 5. Comparing gene numbers

In [8]:
which_assessment = 'gene_number'

genes = [20, 50, 100, 200]

datainfo=0


for genneno in tqdm(genes):
    default_affgenes = int(0.3 * genneno)
    default_n_tfs = int(0.3 * genneno)

    num, div = default_affgenes, 4
    splitaffgenes = [num // div + (1 if x < num % div else 0) for x in range(div)]

    default_perturbeds = [[default_affgenes, 0, 0, 0], 
                  [0, default_affgenes, 0, 0],
                  [0, 0, default_affgenes, 0],
                  [0, 0, 0, default_affgenes],
                  splitaffgenes]
    
    # print(genes, default_affgenes, default_n_tfs, default_perturbeds)
    for p in range(len(default_perturbeds)):

        simulation = GRN_simulator(n_genes=genneno, n_tfs=default_n_tfs, n_samples=default_n_samples,
                                   n_shutdowns=default_perturbeds[p][0], n_inversions=default_perturbeds[p][1], 
                                   n_shifts=default_perturbeds[p][2], n_uncorrelateds=default_perturbeds[p][3], 

                                   proportion_group_case=default_proportion_group_case,
                                   simdata_noise=default_simdata_noise, perturbation_noise=default_perturbation_noise, mean_expression=default_mean_expression,
                                   shift_alpha = default_shift_alpha, ingroup_perturbed_proportion=default_ingroup_perturbed_proportion, zscore_normalize = False)
        simulation.generate_DRN()

        simdrnet_rootname = os.path.join(f'simulations/DR/', which_assessment, f'genenno_{genneno}_perts_{default_perturbeds[p][0]}_{default_perturbeds[p][1]}_{default_perturbeds[p][2]}_{default_perturbeds[p][3]}_')
        #print(simdrnet_rootname)

        # Now, we'll save the object using pickle.
        with open(f'{simdrnet_rootname}.pkl', 'wb') as f:
            pickle.dump(simulation, f)


        simulation.simulated_data.T.to_csv(simdrnet_rootname + '_biomdata.csv')
        simulation.condition_data.to_csv(simdrnet_rootname + '_conddata.csv')
        simulation.refnet.to_csv(simdrnet_rootname + '_refnet.csv')
        simulation.structure.to_csv(simdrnet_rootname + '_structure.csv')
        datainfo += 1
        
print(f"Total number of {which_assessment} simulations: {datainfo}")


100%|███████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.71s/it]

Total number of gene_number simulations: 20


# 6. Comparing in-group heterogeneity

In [9]:
#Baseline parameters
which_assessment = 'ingroup_heterogeneity'

datainfo=0

proportion_case = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for proportion in tqdm(proportion_case):
    for p in range(len(default_perturbeds)):

        #print(perturbeds[p][0], perturbeds[p][1], perturbeds[p][2],  perturbeds[p][3])

        simulation = GRN_simulator(n_genes=default_n_genes, n_tfs=default_n_tfs, n_samples=default_n_samples,
                                   n_shutdowns=default_perturbeds[p][0], n_inversions=default_perturbeds[p][1], 
                                   n_shifts=default_perturbeds[p][2], n_uncorrelateds=default_perturbeds[p][3], 

                                   proportion_group_case=default_proportion_group_case,
                                   simdata_noise=default_simdata_noise, perturbation_noise=default_perturbation_noise, mean_expression=default_mean_expression,
                                   shift_alpha = default_shift_alpha, ingroup_perturbed_proportion=proportion, zscore_normalize = False)
        simulation.generate_DRN()

        simdrnet_rootname = os.path.join(f'simulations/DR/', which_assessment, f'propingroup_{proportion}_perts_{default_perturbeds[p][0]}_{default_perturbeds[p][1]}_{default_perturbeds[p][2]}_{default_perturbeds[p][3]}_')
        #print(simdrnet_rootname)

        # Now, we'll save the object using pickle.
        with open(f'{simdrnet_rootname}.pkl', 'wb') as f:
            pickle.dump(simulation, f)


        simulation.simulated_data.T.to_csv(simdrnet_rootname + '_biomdata.csv')
        simulation.condition_data.to_csv(simdrnet_rootname + '_conddata.csv')
        simulation.refnet.to_csv(simdrnet_rootname + '_refnet.csv')
        simulation.structure.to_csv(simdrnet_rootname + '_structure.csv')
        datainfo += 1

print(f"Total number of {which_assessment} simulations: {datainfo}")



100%|███████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:38<00:00,  4.33s/it]

Total number of ingroup_heterogeneity simulations: 45
